# Py-Píldoras para tratar tu Open Data

Material de trabajo de la Py-Píldora para combatir la desorientación: **GEOCODING**

### 1. Analizar los datos

In [4]:
import pandas as pd
df = pd.read_csv('./201803_detalle.csv', sep=';', encoding = "ISO-8859-1")
df.head(10)

,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO-BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
0,LEVE,AV MONFORTE DE LEMOS 12,3,2018,9.56,90.0,SI,0,SER,"ESTACIONAR, SIN LA CORRESPONDIENTE AUTORIZACIÓ...",,,,
1,GRAVE,AV RAMON Y CAJAL 72,3,2018,11.33,200.0,SI,0,POLICIA MUNICIPAL,"ESTACIONAR EN CARRIL RESERVADO PARA USO, PARAD...",,,,
2,GRAVE,SEGOVIA-TOLEDO,3,2018,5.15,200.0,SI,4,POLICIA MUNICIPAL,REBASAR UN SEMÁFORO EN FASE ROJA. ...,,,,
3,LEVE,AV MONFORTE DE LEMOS 17,3,2018,10.12,90.0,SI,0,SER,"ESTACIONAR, SIN LA CORRESPONDIENTE AUTORIZACIÓ...",,,,
4,LEVE,CIUDADANIA 6,3,2018,14.22,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SEÑA...,,,,
5,LEVE,AV MONFORTE DE LEMOS 8,3,2018,10.18,90.0,SI,0,SER,"ESTACIONAR, SIN LA CORRESPONDIENTE AUTORIZACIÓ...",,,,
6,GRAVE,CL MELCHOR FERNANDEZ ALMAGRO 1,3,2018,10.45,200.0,SI,0,SER,ESTACIONAR EN INTERSECCIÓN. ...,,,,
7,LEVE,CL POETA JOAN MARAGALL 19 B,3,2018,17.12,30.0,SI,0,SER,ESTACIONAR CON AUTORIZACIÓN EN LUGAR HABILITAD...,,,,
8,LEVE,CL VILLA DE MARIN 23,3,2018,11.11,60.0,SI,0,SER,ESTACIONAR CON AUTORIZACIÓN NO VÁLIDA. ...,,,,
9,LEVE,CL VILLA DE MARIN 23,3,2018,14.09,90.0,SI,0,SER,"ESTACIONAR, SIN LA CORRESPONDIENTE AUTORIZACIÓ...",,,,


In [5]:
import pandas as pd

# cargo los datos en un dataframe de pandas
df = pd.read_csv('./201803_detalle.csv', sep=';', 
                 encoding = "ISO-8859-1")

# selecciono los registros sin coordenadas
df_nc = df[df['COORDENADA_X'] == '           ']
with_coords_perc = int(len(df_nc) / len(df) * 100)

# muestro el resultado
print('Total de registros:    {:,}'.format(len(df)))
print('   sin coordenadas:    {:,} -- {}%'
      .format(len(df_nc), with_coords_perc))

Total de registros:    211,325
   sin coordenadas:    157,615 -- 74%


### 2. ¡Truco! Agrupar las direcciones

In [8]:
# número de direcciones únicas sin coordenadas
uniq_address = len(df_nc.LUGAR.unique())

# porcentaje de direcciones únicas frente al total
uniq_perc = int(uniq_address / len(df_nc) * 100)

print('Tengo {:,} direcciones únicas - {}% menos!'
      .format(uniq_address, uniq_perc))

Tengo 32,745 direcciones únicas - 20% menos!


### 3. Cálculos de cuotas en Google Maps

In [13]:
import math

days_needed = math.ceil(len(df_nc) / 2500)
print(f'Necesitaría {len(df_nc)} peticiones / '
      f'{days_needed} días sin agrupar')

new_days_needed = math.ceil(uniq_address / 2500)
print(f'Necesitaría {uniq_address} peticiones / '
      f'{new_days_needed} días si las agrupo')
print(f'\t({len(df_nc) - uniq_address} peticiones / '
      f'{days_needed - new_days_needed} días menos)')

Necesitaría 157615 peticiones / 64 días sin agrupar
Necesitaría 32745 peticiones / 14 días si las agrupo
	(124870 peticiones / 50 días menos)


### 4. Probando datos “en crudo”

In [14]:
import googlemaps
from secrets import GOOGLE_MAPS_KEY

gmaps = googlemaps.Client(key=GOOGLE_MAPS_KEY)

address = 'AV RAMON Y CAJAL 72      '

geocode_result = gmaps.geocode(address)
print(f'{len(geocode_result)} resultados')

10 resultados


In [15]:
pretty_results = [result['formatted_address'] 
                  for result in geocode_result]
print('\n'.join(pretty_results))

Ramón y Cajal Etorb., 72, 48014 Bilbo, Bizkaia, Spain
Av. Ramón y Cajal, 72, 05480 Candeleda, Ávila, Spain
Av. Ramón y Cajal, 72, 10612 Jerte, Cáceres, Spain
Av. Ramón y Cajal, 72, 47230 Matapozuelos, Valladolid, Spain
Av. Ramón y Cajal, 72, 13400 Almadén, Cdad. Real, Spain
Av. Ramón y Cajal, 72, 46470 Catarroja, Valencia, Spain
Av. Ramón y Cajal, 72, 30530 Cieza, Murcia, Spain
Av. Ramon y Cajal, 72, 41658 Martín de la Jara, Sevilla, Spain
Av. Ramón y Cajal, 72, 21720 Rociana del Condado, Huelva, Spain
Av. Ramón y Cajal, 72, 46870 Ontinyent, Valencia, Spain


### 4. Preparar las direcciones

Prepara la dirección para hacerselo "más fácil" al geocoder
1. añado la ciudad (Madrid) y el país (Spain) a la dirección
2. elimino los espacios en blanco innecesarios
3. reemplazo los guiones bajos y "SN" por espacios en blanco

In [17]:
address = 'AV RAMON Y CAJAL 72'

# elimina el texto "conflictivo"
address = address.strip().replace('_', ' ').replace('SN', ' ')

# añade la ciudad y el país
address = '{}, Madrid, Spain'.format(address)
print('Geocoding -- {}'.format(address))

Geocoding -- AV RAMON Y CAJAL 72, Madrid, Spain


In [18]:
geocode_result = gmaps.geocode(address)
result_count = len(geocode_result)
print(f'{result_count} resultados')

pretty_results = [result['formatted_address'] 
                  for result in geocode_result]
print('\n'.join(pretty_results))

1 resultados
Av. de Ramón y Cajal, 72, 28002 Madrid, Spain


### 4. Lanzar el proceso

In [43]:
for address in df_nc.LUGAR.unique()[:5]:
    # formatea la dirección
    address = '{}, Madrid, Spain'.format(address.strip()
                                         .replace('_', ' ')
                                         .replace('SN', ' '))
    # llama al servicio de geocodificación
    geocode_result = gmaps.geocode(address)
    
    # muestra el resultado
    result = geocode_result[0]
    print(f'{address}')
    print(f"\t{result['geometry']['location']}\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain
	{'lat': 40.4797213, 'lng': -3.6960687}

AV RAMON Y CAJAL 72, Madrid, Spain
	{'lat': 40.4522, 'lng': -3.66439}

SEGOVIA-TOLEDO, Madrid, Spain
	{'lat': 40.416572, 'lng': -3.702082}

AV MONFORTE DE LEMOS 17, Madrid, Spain
	{'lat': 40.4795744, 'lng': -3.6966646}

CIUDADANIA 6, Madrid, Spain
	{'lat': 40.35638, 'lng': -3.69231}



## Pruebas de geocoding con otras librerías
(no incluído en las slides de la presentación)

### [Geocoder](https://github.com/DenisCarriere/geocoder)

Usando el geocoder de Google

In [69]:
import geocoder

for address in df_nc.LUGAR.unique()[:10]:
    address = '{}, Madrid, Spain'.format(address.strip().replace('_', ' ').replace('SN', ' '))
    geocoder_result = geocoder.google(address)
    print(f"{address} -> {geocoder_result.address} - {geocoder_result.latlng}\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain -> None - None

AV RAMON Y CAJAL 72, Madrid, Spain -> Av. de Ramón y Cajal, 72, 28002 Madrid, Spain - [40.4522, -3.66439]

SEGOVIA-TOLEDO, Madrid, Spain -> 2º, Carrera de S. Jerónimo, 15, 28014 Madrid, Spain - [40.416572, -3.702082]

AV MONFORTE DE LEMOS 17, Madrid, Spain -> None - None

CIUDADANIA 6, Madrid, Spain -> None - None

AV MONFORTE DE LEMOS 8, Madrid, Spain -> None - None

CL MELCHOR FERNANDEZ ALMAGRO 1, Madrid, Spain -> None - None

CL POETA JOAN MARAGALL 19 B, Madrid, Spain -> Calle del Poeta Joan Maragall, 19B, 28020 Madrid, Spain - [40.4572902, -3.6923661]

CL VILLA DE MARIN 23, Madrid, Spain -> Calle Villa de Marin, 23, 28029 Madrid, Spain - [40.4765385, -3.7003378]

CL PINTOR FRANCISCO LLORENS 2, Madrid, Spain -> None - None



Usando el geocoder de ArcGIS

In [70]:
for address in df_nc.LUGAR.unique()[:10]:
    address = '{}, Madrid, Spain'.format(address.strip().replace('_', ' ').replace('SN', ' '))
    geocoder_result = geocoder.arcgis(address)
    print(f"{address} -> {geocoder_result.address} - {geocoder_result.latlng}\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain -> Avenida de Monforte de Lemos 12, 28029, La Paz, Madrid, Comunidad de Madrid - [40.47964141271852, -3.696078850539554]

AV RAMON Y CAJAL 72, Madrid, Spain -> Avenida de Ramón y Cajal 72, 28043, San Juan Bautista, Madrid, Comunidad de Madrid - [40.45310011070236, -3.66302383267301]

SEGOVIA-TOLEDO, Madrid, Spain -> Toledo, Castilla-La Mancha - [39.79377747800004, -4.147973436999962]

AV MONFORTE DE LEMOS 17, Madrid, Spain -> Avenida de Monforte de Lemos 17, 28029, La Paz, Madrid, Comunidad de Madrid - [40.479279539529266, -3.6963352559004865]

CIUDADANIA 6, Madrid, Spain -> Calle de la Ciudadanía 6, 28041, Los Rosales, Madrid, Comunidad de Madrid - [40.356404528997686, -3.692240664723869]

AV MONFORTE DE LEMOS 8, Madrid, Spain -> Avenida de Monforte de Lemos 8, 28029, La Paz, Madrid, Comunidad de Madrid - [40.47933541271853, -3.692928599082482]

CL MELCHOR FERNANDEZ ALMAGRO 1, Madrid, Spain -> Calle de Melchor Fernández Almagro 1, 28029, La Paz, 

Usando el geocoder de Open Street Maps

In [72]:
for address in df_nc.LUGAR.unique()[:10]:
    address = '{}, Madrid, Spain'.format(address.strip().replace('_', ' ').replace('SN', ' '))
    geocoder_result = geocoder.osm(address)
    print(f"{address} -> {geocoder_result.address} - {geocoder_result.latlng}\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain -> None - None

AV RAMON Y CAJAL 72, Madrid, Spain -> Avenida Ramón y Cajal, Dehesa Boyal, San Sebastián de los Reyes, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28108, España - [40.5420219, -3.6183706]

SEGOVIA-TOLEDO, Madrid, Spain -> Ronda de Segovia - Pta. Toledo, Ronda de Segovia, Imperial, Centro, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28001, España - [40.4073078, -3.7130116]

AV MONFORTE DE LEMOS 17, Madrid, Spain -> None - None

CIUDADANIA 6, Madrid, Spain -> Calle de la Ciudadanía, El Espinillo, Villaverde, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28001, España - [40.3566265, -3.6910722]

AV MONFORTE DE LEMOS 8, Madrid, Spain -> None - None

CL MELCHOR FERNANDEZ ALMAGRO 1, Madrid, Spain -> None - None

CL POETA JOAN MARAGALL 19 B, Madrid, Spain -> None - None

CL VILLA DE MARIN 23, Madrid, Spain -> None - None

CL PINTOR FRAN

## [Geopy](https://geopy.readthedocs.io/en/stable/)

Usando el geocoder de Google

In [23]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3()

for address in df_nc.LUGAR.unique()[:10]:
    address = '{}, Madrid, Spain'.format(address.strip().replace('_', ' ').replace('SN', ' '))
    location = geolocator.geocode(address)
    if location:
        print(f"{address} -> {location.address} - {location.latitude}, {location.longitude}\n")
    else:
        print(f"{address} -> Not found\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain -> Av. de Monforte de Lemos, 12, 28029 Madrid, Spain - 40.4797213, -3.6960687

AV RAMON Y CAJAL 72, Madrid, Spain -> Av. de Ramón y Cajal, 72, 28002 Madrid, Spain - 40.4522, -3.66439

SEGOVIA-TOLEDO, Madrid, Spain -> 2º, Carrera de S. Jerónimo, 15, 28014 Madrid, Spain - 40.416572, -3.702082

AV MONFORTE DE LEMOS 17, Madrid, Spain -> Av. de Monforte de Lemos, 17, 28029 Madrid, Spain - 40.4795744, -3.6966646

CIUDADANIA 6, Madrid, Spain -> Calle Ciudadanía, 6, 28041 Madrid, Spain - 40.35638, -3.69231



GeocoderQuotaExceeded: The given key has gone over the requests limit in the 24 hour period or has submitted too many requests in too short a period of time.

Usando el geocoder de ArcGIS

In [24]:
from geopy.geocoders import ArcGIS
geolocator = ArcGIS()

for address in df_nc.LUGAR.unique()[:10]:
    address = '{}, Madrid, Spain'.format(address.strip().replace('_', ' ').replace('SN', ' '))
    location = geolocator.geocode(address)
    if location:
        print(f"{address} -> {location.address} - {location.latitude}, {location.longitude}\n")
    else:
        print(f"{address} -> Not found\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain -> Avenida de Monforte de Lemos 12, 28029, La Paz, Madrid, Comunidad de Madrid - 40.47964141271852, -3.696078850539554

AV RAMON Y CAJAL 72, Madrid, Spain -> Avenida de Ramón y Cajal 72, 28043, San Juan Bautista, Madrid, Comunidad de Madrid - 40.45310011070236, -3.66302383267301

SEGOVIA-TOLEDO, Madrid, Spain -> Toledo, Castilla-La Mancha - 39.79377747800004, -4.147973436999962

AV MONFORTE DE LEMOS 17, Madrid, Spain -> Avenida de Monforte de Lemos 17, 28029, La Paz, Madrid, Comunidad de Madrid - 40.479279539529266, -3.6963352559004865

CIUDADANIA 6, Madrid, Spain -> Calle de la Ciudadanía 6, 28041, Los Rosales, Madrid, Comunidad de Madrid - 40.356404528997686, -3.692240664723869

AV MONFORTE DE LEMOS 8, Madrid, Spain -> Avenida de Monforte de Lemos 8, 28029, La Paz, Madrid, Comunidad de Madrid - 40.47933541271853, -3.692928599082482

CL MELCHOR FERNANDEZ ALMAGRO 1, Madrid, Spain -> Calle de Melchor Fernández Almagro 1, 28029, La Paz, Madrid, Comu

Usando el geocoder de Open Street Maps (Nominatim)

In [22]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

for address in df_nc.LUGAR.unique()[:10]:
    address = '{}, Madrid, Spain'.format(address.strip().replace('_', ' ').replace('SN', ' '))
    location = geolocator.geocode(address)
    if location:
        print(f"{address} -> {location.address} - {location.latitude}, {location.longitude}\n")
    else:
        print(f"{address} -> Not found\n")

AV MONFORTE DE LEMOS 12, Madrid, Spain -> Not found

AV RAMON Y CAJAL 72, Madrid, Spain -> Avenida Ramón y Cajal, Dehesa Boyal, San Sebastián de los Reyes, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28108, España - 40.5420219, -3.6183706

SEGOVIA-TOLEDO, Madrid, Spain -> Ronda de Segovia - Pta. Toledo, Ronda de Segovia, Imperial, Centro, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28001, España - 40.4073078, -3.7130116

AV MONFORTE DE LEMOS 17, Madrid, Spain -> Not found

CIUDADANIA 6, Madrid, Spain -> Calle de la Ciudadanía, El Espinillo, Villaverde, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28001, España - 40.3566265, -3.6910722

AV MONFORTE DE LEMOS 8, Madrid, Spain -> Not found

CL MELCHOR FERNANDEZ ALMAGRO 1, Madrid, Spain -> Not found

CL POETA JOAN MARAGALL 19 B, Madrid, Spain -> Not found

CL VILLA DE MARIN 23, Madrid, Spain -> Not found

CL PINTOR FRANCISCO LLORENS 2, M